# OCT 1, 2025 - Tinkering with Data Analysis

#### Some ideas:
* response time (b/w each "category" of petitions, individuals vs big companies,   
* count of big companies vs individuals
* type of requests being made
* success rate: type of requests being denied vs approved
* what specifically gets a petition denied/approved?

In [26]:
%pip install seaborn
%pip install pandas
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [28]:
petitions_2017 = pd.read_csv("data/2017_PETITIONS_COMPLETE_071825_MP.xlsx - Sheet1.csv")
responses_2017 = pd.read_csv("data/2017_RESPONSES_COMPLETE_071825_MP.xlsx - Sheet1.csv")
pharma_comps = pd.read_csv("data/WR-Issued-June-2024-Approved-Active-Moiety-6-27-24.csv")

In [29]:
petitions_2017.sort_values("Identity of Submitting Entity").head(3)

,File Name,Date of Petition,Date Comments,Identity of Submitting Entity,Representation Details,Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request,Status of Review,Review Comments
34,FDA-2017-P-1251-0001_Citizen_Petition_from_Ame...,2/27/2017,NaN,- American Feed Industry Association (AFIA)\n-...,Not mentioned.,"- 21 U.S.C. §§ 351(a)(2)(B), 360b(m), 371(a), ...",The application of 21 C.F.R. Part 11 (electron...,- Exempt all medicated feed cGMP-related recor...,- High cost and burden of Part 11 compliance (...,False,NaN
138,FDA-2017-P-4372-0002_Citizen_Petition_from_AEG...,NaN,No date recognized by ChatGPT nor apparent thr...,"AEGIS Regulatory, Inc.",Not mentioned.,- 21 C.F.R. § 10.30 (Citizen Petition)\n- 21 C...,Enforcement and importation practices regardin...,- Prohibit the entry of adulterated cosmetic p...,- Reports of adverse health effects including ...,False,NaN
52,FDA-2017-P-1990-0001_Citizen_Petition_from_AID...,3/23/2017,"Letterhead dated March 23, but stamp reflects ...","AIDP, Inc.\n",Not mentioned.,- 21 C.F.R. § 10.30\n- 21 C.F.R. § 101.9(c)(6)...,Definition of dietary fiber under the revised ...,Amend 21 C.F.R. § 101.9(c)(6)(i) to include xy...,AIDP presented evidence that XOS provides mult...,False,NaN


In [30]:
responses_2017.head(3)

,File Name,Date of Response,Date Comments,Responding FDA Center,Response to Petition,Cited Statutes or Regulations,Justification for Response,Status of Review,Review Comments
0,FDA-2017-P-0052-0150_Interim_Response_from_CBE...,NaN,"No date recognized by ChatGPT, but found in si...",Center for Biologics Evaluation and Research (...,Interim Response,21 CFR 10.30(e)(2) (FDA regulations on citizen...,"The FDA cites ""existence of other FDA prioriti...",False,NaN
1,FDA-2017-P-0052-0203_Response_letter_from_FDA_...,NaN,"No date recognized by ChatGPT, but found in si...",Center for Biologics Evaluation and Research (...,Denied,- Section 361 of the Public Health Service Act...,The FDA denied the petition based on the follo...,False,NaN
2,FDA-2017-P-0115-0007_Memorandum_of_suitability...,6/12/2023,NaN,Center for Drug Evaluation and Research (CDER),Withdrawn,Not applicable.,- FDA contacted the petitioner (Jubilant Gener...,False,NaN


In [31]:
pharma_comps.head()

,Active Moiety,Sponsor,Date Issued,Unnamed: 3,Unnamed: 4
0,Abacavir,"Glaxo Wellcome, Inc.",8/20/98,NaN,NaN
1,Abacavir/Dolutegravir/Lamivudine,ViiV Healthcare Company,5/16/18,NaN,NaN
2,Abatacept,Bristol-Myers Squibb Company,9/13/13,NaN,NaN
3,Abemaciclib,Eli Lilly & Co.,6/7/19,NaN,NaN
4,Acamprosate,Forest Labs,11/9/05,NaN,NaN


## Feature Engineering

### Labeling the submitters

#### Cleaning "Identity of Submitting Entity"
* petitions filed by individuals usually labeled by:
    * "Individual citizen petition..."
    * "Multiple co-signatories:..."
    * "Public Citizen" but be careful with "Public Citizen’s Health Research Group"
    * specific names first and last (but whole title not ending in LLP, LLC, Inc, P.C; also be careful with companies having "normal" names at the beginning like wilson, kleinfeld, but dont end in llp, llc etc)
    * "Anonymous (redacted)" (?)
      
    * ***individual: 'individual citizen', 'multiple co-signatories', 'anonymous', ' md', ' jd', ' pharmd', ' ph.d'*** 

* petitions filed by corps usually labeled by:
    * ***industry/corporate: 'llc', 'inc', 'pharma', 'ltd', limited', 'corp', 'laboratories', 'technologies'***

    * ***law/consulting: 'llp', 'p.c.', 'law', 'legal', 'consultant', 'regulatory', 'associates'***

    * ***advocacy/academic: 'association', 'university', 'center for', 'society', 'foundation'***


#### What to clean
* "\n", "\t" at the end of some
* different variations
  * 'Kleinfeld, Kaplan & Becker LLP',
  * 'Kleinfeld, Kaplan & Becker, LLP',
  * 'Kleinfeld, Kaplan and Becker LLP\t',
  * 'Kleinfeld, Kaplan and Becker, LLP\n'

  * 'Apotex Inc.\n',
  * 'Apotex Inc. and Apobiologix (a Division of Apotex Inc.) \n'
    
  * 'INC Research, LLC',
  * 'INC Research/inVentiv Health'
* multiple corporations in single cell separated by dashes - or "and"
  * '- American Feed Industry Association (AFIA)\n- National Grain and Feed Association (NGFA)'
  * 'Abhai, LLC and KVK-Tech, Inc.'
* edge cases:
  * 'Kent Heckenlively, JD; also signed by Rima E. Laibow, MD and Ralph Fucetola, JD on behalf of Natural Solutions Foundation and Institute for Health Research respectively \n'
  * 'NJOY, along with:\n- Vapor Technology Association\n- SFATA\n- National Association of Tobacco Outlets, Inc.\n- CITMA\n- Turning Point Brands, Inc.\n- Mistic Electronic Cigarettes\n- Johnson Creek Vapor Company\n- Nicopure Labs LLC\n- Five Pawns Inc.\n- Gaiatrend USA\n- Glas, LLC\n- Shift Ventures, Inc.\n- E-Alternative Solutions, Inc.\n- Saffire Vapor'

In [32]:
#petitions_2017["Identity of Submitting Entity"].sort_values().unique()

In [33]:
# cleaning "Identity of Submitting Entity"
petitions_2017_mod = petitions_2017.copy()

# remove rows w/ no submitting entity
petitions_2017_mod = petitions_2017_mod.dropna(subset = ["Identity of Submitting Entity"])

# removing "\n", "\t", "\n-", "\n•\u202f"
petitions_2017_mod["Submitter"] = petitions_2017_mod["Identity of Submitting Entity"].astype(str)
petitions_2017_mod["Submitter"] = petitions_2017_mod["Submitter"].str.strip().str.replace("\n-", " ").str.replace("\n•\u202f", " ").str.replace("\u202f", " ")

#petitions_2017["Submitter"].sort_values().unique()

In [34]:
# cleaning pharmaceutical companies corpus

# drop last row bc NaN
pharma_comps = pharma_comps.drop(len(pharma_comps) - 1)

# remove inc, lp, etc
def normalize_company_name(name):
    name_lower = name.lower()
    
    # looks for common suffixes (inc, llc, etc.) and removes them from the end of the comp name
    name_lower = re.sub(r'(\s*,\s*|\s+)(inc|llc|l\.p|lp|limited|corp|co|ltd)\.?\s*$', '', name_lower)
    
    # remove any remaining punctuation and extra whitespace
    name_lower = re.sub(r'[^\w\s]', '', name_lower)
    name_lower = name_lower.strip()
    
    return name_lower
    
pharma_names_list = pharma_comps["Sponsor"].str.lower().str.strip().sort_values().unique()
#display(pharma_names_list)

pharma_names_norm = pharma_comps["Sponsor"].apply(normalize_company_name).sort_values().unique()
#display(pharma_names_norm.head)

In [35]:
#if any(word in 'some one long two phrase three' for word in list_):
# list of pharma companies for categorizing https://share.google/Lzg1nML50MNBjpQkP

# categorize each submitter
individual_terms = ['individual', 'citizen', 'individual citizen', 'multiple co-signatories', ' md', ' jd', ' pharmd', ' ph.d', ' senator']
indus_corp_terms = ['llc', 'inc', 'pharma', 'ltd', 'limited', 'corp', 'laboratories', 'technologies']
outlier_indus = ['unilever']
law_consult_terms = ['llp', 'p.c.', 'p.c', 'law', 'legal', 'consultant', 'consulting', 'regulatory', 'associates', 'solution']
advoc_academic_terms = ['association', 'university', 'center for', 'society', 'foundation', 'research group', 'research', 
                        'lab', 'registry', 'institute', 'council', 'choice']

def categorize_submitter(submitter_title):
    submitter_title_lower = submitter_title.lower()
    #submitter_title_norm = normalize_company_name(submitter_title)
    
    if pd.isna(submitter_title) or "anonymous" in submitter_title_lower:
        return "other"
        
    elif submitter_title_lower in pharma_names_list or submitter_title_lower in pharma_names_norm:
        return "industry/corporation"
        
    # for each identifier word in individual, industry, etc, check if that word exists in the title
    if submitter_title_lower in outlier_indus or any(word in submitter_title_lower for word in indus_corp_terms):
        return "industry/corporation"
        
    elif any(word in submitter_title_lower for word in law_consult_terms):
        return "law/consulting"
        
    elif any(word in submitter_title_lower for word in advoc_academic_terms):
        return "advocacy/academic"
        
    elif any(word in submitter_title_lower for word in individual_terms):
        return "individual"

    # check for short individual's names (usually first, middle, last)
    elif len(submitter_title_lower.split()) <= 3:
        return "individual"

    else:
        return "other"

In [36]:
petitions_2017_mod["Submitter Type"] = petitions_2017_mod["Submitter"].apply(categorize_submitter)
#petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "law/consulting"]

petitions_2017_mod["Submitter Type"].value_counts()

Submitter Type
industry/corporation    128
law/consulting           78
advocacy/academic        15
individual               13
other                     2
Name: count, dtype: int64

In [37]:
display(petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "advocacy/academic"][["Submitter", "Submitter Type"]])
display(petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "other"][["Submitter", "Submitter Type"]])
display(petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "individual"][["Submitter", "Submitter Type"]])

,Submitter,Submitter Type
0,Donor Sibling Registry,advocacy/academic
5,Malaysian Rubber Export Promotion Council (MREPC),advocacy/academic
11,"Union of Concerned Scientists, Center for Scie...",advocacy/academic
34,- American Feed Industry Association (AFIA) N...,advocacy/academic
38,The Good Food Institute (GFI),advocacy/academic
66,Consumers for Dental Choice,advocacy/academic
75,The Tatia Oden French Memorial Foundation,advocacy/academic
103,Salter Labs,advocacy/academic
146,American Dental Association (ADA),advocacy/academic
152,Wilson Disease Association (WDA),advocacy/academic


,Submitter,Submitter Type
98,Anonymous (redacted),other
100,Anonymous (redacted),other


,Submitter,Submitter Type
21,Public Citizen,individual
22,David Wong,individual
69,Senator Charles E. Schumer and Senator Kirsten...,individual
92,Individual citizen petition submitted jointly ...,individual
93,"Philip Spiller, along with: George A. Gellert...",individual
102,Michael Boggi,individual
133,William Bonificio,individual
151,"Chul-Hi Park, Ph.D.",individual
163,"Dennis L. Ryll, MD",individual
164,"Dennis L. Ryll, MD",individual


In [38]:
# download to check bc too many rows
filter_industry = petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "industry/corporation"][["Submitter", "Submitter Type"]]
filter_industry.to_csv("industry_check.csv", index=False)

filter_law = petitions_2017_mod[petitions_2017_mod["Submitter Type"] == "law/consulting"][["Submitter", "Submitter Type"]]
filter_law.to_csv("law_check.csv", index=False)

In [39]:
petitions_2017_mod.head(3)

,File Name,Date of Petition,Date Comments,Identity of Submitting Entity,Representation Details,Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request,Status of Review,Review Comments,Submitter,Submitter Type
0,FDA-2017-P-0052-0001_Citizen_Petition_from_Don...,1/1/2017,NaN,Donor Sibling Registry\n,Not mentioned.,- 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...,Lack of regulation and oversight in the sperm ...,The petitioner asks the FDA Commissioner to in...,The petition outlines that the U.S. cryobank i...,False,NaN,Donor Sibling Registry,advocacy/academic
1,FDA-2017-P-0115-0001_Citizen_Petition_from_Jub...,12/22/2016,NaN,Jubilant Generics Limited,Not mentioned.,- Section 505(j)(2)(C) of the FD&C Act [21 U.S...,FDA’s requirement to approve an ANDA petition ...,FDA is requested to determine that Levetiracet...,- The proposed product matches the RLD in conc...,False,NaN,Jubilant Generics Limited,industry/corporation
2,FDA-2017-P-0137-0001_Citizen_Petition_from_Mac...,1/9/2017,NaN,Macleods Pharmaceuticals Limited,Not mentioned.,"- Section 505(j) of the Federal Food, Drug, an...",The petition comments on the absence of a curr...,Macleods requests FDA to designate the approve...,- The current RLD (Pyrazinamide 500 mg by Dava...,False,NaN,Macleods Pharmaceuticals Limited,industry/corporation


### Combining petitions and responses data 

In [40]:
responses_2017.head(3)

,File Name,Date of Response,Date Comments,Responding FDA Center,Response to Petition,Cited Statutes or Regulations,Justification for Response,Status of Review,Review Comments
0,FDA-2017-P-0052-0150_Interim_Response_from_CBE...,NaN,"No date recognized by ChatGPT, but found in si...",Center for Biologics Evaluation and Research (...,Interim Response,21 CFR 10.30(e)(2) (FDA regulations on citizen...,"The FDA cites ""existence of other FDA prioriti...",False,NaN
1,FDA-2017-P-0052-0203_Response_letter_from_FDA_...,NaN,"No date recognized by ChatGPT, but found in si...",Center for Biologics Evaluation and Research (...,Denied,- Section 361 of the Public Health Service Act...,The FDA denied the petition based on the follo...,False,NaN
2,FDA-2017-P-0115-0007_Memorandum_of_suitability...,6/12/2023,NaN,Center for Drug Evaluation and Research (CDER),Withdrawn,Not applicable.,- FDA contacted the petitioner (Jubilant Gener...,False,NaN


In [41]:
responses_2017_mod = responses_2017.drop(["Status of Review", "Review Comments"], axis = "columns")
#responses_2017_mod

In [42]:
# use numbers after "FDA-2017-P-..." as unique id
petitions_2017_mod["File ID"] = petitions_2017_mod["File Name"].str.extract(r'(FDA-\d{4}-P-\d{4})')
display(petitions_2017_mod.head(3))
responses_2017_mod["File ID"] = responses_2017_mod["File Name"].str.extract(r'(FDA-\d{4}-P-\d{4})')
display(responses_2017_mod.head(3))

,File Name,Date of Petition,Date Comments,Identity of Submitting Entity,Representation Details,Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request,Status of Review,Review Comments,Submitter,Submitter Type,File ID
0,FDA-2017-P-0052-0001_Citizen_Petition_from_Don...,1/1/2017,NaN,Donor Sibling Registry\n,Not mentioned.,- 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...,Lack of regulation and oversight in the sperm ...,The petitioner asks the FDA Commissioner to in...,The petition outlines that the U.S. cryobank i...,False,NaN,Donor Sibling Registry,advocacy/academic,FDA-2017-P-0052
1,FDA-2017-P-0115-0001_Citizen_Petition_from_Jub...,12/22/2016,NaN,Jubilant Generics Limited,Not mentioned.,- Section 505(j)(2)(C) of the FD&C Act [21 U.S...,FDA’s requirement to approve an ANDA petition ...,FDA is requested to determine that Levetiracet...,- The proposed product matches the RLD in conc...,False,NaN,Jubilant Generics Limited,industry/corporation,FDA-2017-P-0115
2,FDA-2017-P-0137-0001_Citizen_Petition_from_Mac...,1/9/2017,NaN,Macleods Pharmaceuticals Limited,Not mentioned.,"- Section 505(j) of the Federal Food, Drug, an...",The petition comments on the absence of a curr...,Macleods requests FDA to designate the approve...,- The current RLD (Pyrazinamide 500 mg by Dava...,False,NaN,Macleods Pharmaceuticals Limited,industry/corporation,FDA-2017-P-0137


,File Name,Date of Response,Date Comments,Responding FDA Center,Response to Petition,Cited Statutes or Regulations,Justification for Response,File ID
0,FDA-2017-P-0052-0150_Interim_Response_from_CBE...,NaN,"No date recognized by ChatGPT, but found in si...",Center for Biologics Evaluation and Research (...,Interim Response,21 CFR 10.30(e)(2) (FDA regulations on citizen...,"The FDA cites ""existence of other FDA prioriti...",FDA-2017-P-0052
1,FDA-2017-P-0052-0203_Response_letter_from_FDA_...,NaN,"No date recognized by ChatGPT, but found in si...",Center for Biologics Evaluation and Research (...,Denied,- Section 361 of the Public Health Service Act...,The FDA denied the petition based on the follo...,FDA-2017-P-0052
2,FDA-2017-P-0115-0007_Memorandum_of_suitability...,6/12/2023,NaN,Center for Drug Evaluation and Research (CDER),Withdrawn,Not applicable.,- FDA contacted the petitioner (Jubilant Gener...,FDA-2017-P-0115


In [43]:
# FINALIZING petitions data
# adding withdrawals as a feature (separate column), so file id can be unique
petitions_2017_mod["Withdrawal File"] = petitions_2017_mod["File Name"].str.lower().str.contains("withdraw")
withdrawals_id = petitions_2017_mod[petitions_2017_mod["Withdrawal File"] == True]["File ID"]
petitions_2017_mod['Was_Withdrawn'] = petitions_2017_mod["File ID"].isin(withdrawals_id)

# cleaning
petitions_2017_main = petitions_2017_mod[petitions_2017_mod["Withdrawal File"] == False].copy()
petitions_2017_main = petitions_2017_main.drop(["Identity of Submitting Entity", "Withdrawal File", "Status of Review", "File Name", 
                                                "Date Comments", "Representation Details", "Review Comments"], axis = "columns")
petitions_2017_main = petitions_2017_main.rename(columns = {"Cited Statutes or Regulations": "Petition Cited Statutes or Regulations"})
new_column_order = ['File ID', 'Date of Petition', 'Was_Withdrawn', 'Submitter', 'Submitter Type',  
                    'Petition Cited Statutes or Regulations', 'FDA Action Commented On', 'Requested Action', 'Justification for Request']

petitions_2017_main = petitions_2017_main[new_column_order]

petitions_2017_main.head(3)
petitions_2017_main.to_csv("petitions_2017_main.csv", index=False)


In [44]:
# final decision for each petition (ignoring interim)
final_decisions = responses_2017_mod[~responses_2017_mod["Response to Petition"].str.lower().str.contains("interim")].groupby("File ID")["Response to Petition"].first()
#final_decisions

# count of interim responses for each file id
interim_counts = responses_2017_mod[responses_2017_mod["Response to Petition"].str.lower().str.contains("interim")].groupby("File ID").size()
#interim_counts

# date of interim & very last response for each file id
final_decision_dates = responses_2017_mod.groupby("File ID")["Date of Response"].max()
interim_dates = responses_2017_mod[responses_2017_mod["Response to Petition"].str.lower().str.contains("interim")].groupby("File ID")["Date of Response"].first()

# responding FDA center
fda_center = responses_2017_mod.groupby("File ID")["Responding FDA Center"].first()

# cited statutes or regulations (ignoring interim)
statutes_regs = responses_2017_mod[~responses_2017_mod["Response to Petition"].str.lower().str.contains("interim")].groupby("File ID")["Cited Statutes or Regulations"].first()

# justification for interim response 
just_interim = responses_2017_mod[responses_2017_mod["Response to Petition"].str.lower().str.contains("interim")].groupby("File ID")["Justification for Response"].first()

# justification for final response
just_final = responses_2017_mod[~responses_2017_mod["Response to Petition"].str.lower().str.contains("interim")].groupby("File ID")["Justification for Response"].first()

# new responses df
responses_agg = pd.DataFrame({
    "Final Decision": final_decisions,
    "Number of Interim Responses": interim_counts,
    "Interim Response Date": interim_dates,
    "Last Response Date": final_decision_dates,
    "Responding FDA Center": fda_center,
    "Responses Cited Statutes or Regulations": statutes_regs,
    "Justification for Interim Response": just_interim,
    "Justification for Final Response": just_final
}).reset_index()

# fill in 0 for petitions that had a final decision but no interim responses
responses_agg["Number of Interim Responses"] = responses_agg["Number of Interim Responses"].fillna(0)
#responses_agg["Interim_Response_Date"] = responses_agg["Interim_Response_Date"].fillna(np.NaN)

# merging into FINAL DF!
final_df = pd.merge(petitions_2017_main, responses_agg, on = "File ID", how = "left")

display(responses_agg.head(3))
display(final_df.head(3))

# download the final version
final_df.to_csv("final_version_petitions_and_responses.csv", index=False)

,File ID,Final Decision,Number of Interim Responses,Interim Response Date,Last Response Date,Responding FDA Center,Responses Cited Statutes or Regulations,Justification for Interim Response,Justification for Final Response
0,FDA-2017-P-0052,Denied,1.0,None,NaN,Center for Biologics Evaluation and Research (...,- Section 361 of the Public Health Service Act...,"The FDA cites ""existence of other FDA prioriti...",The FDA denied the petition based on the follo...
1,FDA-2017-P-0115,Withdrawn,0.0,NaN,6/12/2023,Center for Drug Evaluation and Research (CDER),Not applicable.,NaN,- FDA contacted the petitioner (Jubilant Gener...
2,FDA-2017-P-0137,NaN,1.0,7/7/2017,7/7/2017,Center for Drug Evaluation and Research (CDER),NaN,"The FDA stated that the petition ""raises compl...",NaN


,File ID,Date of Petition,Was_Withdrawn,Submitter,Submitter Type,Petition Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request,Final Decision,Number of Interim Responses,Interim Response Date,Last Response Date,Responding FDA Center,Responses Cited Statutes or Regulations,Justification for Interim Response,Justification for Final Response
0,FDA-2017-P-0052,1/1/2017,False,Donor Sibling Registry,advocacy/academic,- 21 C.F.R. § 10.20\n- 21 C.F.R. § 10.30\n- 21...,Lack of regulation and oversight in the sperm ...,The petitioner asks the FDA Commissioner to in...,The petition outlines that the U.S. cryobank i...,Denied,1.0,None,NaN,Center for Biologics Evaluation and Research (...,- Section 361 of the Public Health Service Act...,"The FDA cites ""existence of other FDA prioriti...",The FDA denied the petition based on the follo...
1,FDA-2017-P-0115,12/22/2016,False,Jubilant Generics Limited,industry/corporation,- Section 505(j)(2)(C) of the FD&C Act [21 U.S...,FDA’s requirement to approve an ANDA petition ...,FDA is requested to determine that Levetiracet...,- The proposed product matches the RLD in conc...,Withdrawn,0.0,NaN,6/12/2023,Center for Drug Evaluation and Research (CDER),Not applicable.,NaN,- FDA contacted the petitioner (Jubilant Gener...
2,FDA-2017-P-0137,1/9/2017,True,Macleods Pharmaceuticals Limited,industry/corporation,"- Section 505(j) of the Federal Food, Drug, an...",The petition comments on the absence of a curr...,Macleods requests FDA to designate the approve...,- The current RLD (Pyrazinamide 500 mg by Dava...,NaN,1.0,7/7/2017,7/7/2017,Center for Drug Evaluation and Research (CDER),NaN,"The FDA stated that the petition ""raises compl...",NaN


# OCT 15, 2025 - More Cleaning

## Add column for year, impute censored data for the 